In [1]:
import json
import cv2
import shutil
import torch
import torchvision
from tqdm import tqdm

import numpy as np
import torchvision.transforms as T
import seaborn as sns
import torch.nn.functional as F

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models
from torch import nn, optim
from torch.optim import lr_scheduler
from tqdm import tqdm
from matplotlib.ticker import MaxNLocator

from matplotlib import pyplot as plt
from glob import glob
from pathlib import Path
from collections import defaultdict
from os import listdir
from os.path import isfile, join
import PIL.Image as Image


def load_json(filename):
    with open(filename,mode="r") as f:
        return json.load(f)

def save_json(data,filename):
    with open(filename, mode="w") as f:
        json.dump(data,f)
    return

In /home/nlplab9/anaconda3/envs/dstc_cyj/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/nlplab9/anaconda3/envs/dstc_cyj/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/nlplab9/anaconda3/envs/dstc_cyj/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/nlplab9/anaconda3/envs/dstc_cyj/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.


In [2]:
# remove data
# !rm -rf vision_data/color

In [3]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [4]:
from pathlib import Path
cwd = Path().resolve()

In [5]:
# scene_folder = "simmc2_scene_jsons_dstc10_public"
scene_folder = "simmc2_scene_jsons_dstc10_teststd"
scenes = sorted(glob(str(scene_folder + "/*_scene.json")))
bboxes = sorted(glob(str(scene_folder + "/*_bbox.json")))

image_path1=str(cwd)+  "/simmc2_scene_images_dstc10_public_part2"
image_path2=str(cwd)+ "/simmc2_scene_images_dstc10_public_part1"
image_path_test =str(cwd)+ "/simmc2_scene_images_dstc10_teststd"
image_files1 = [f for f in listdir(image_path1) if isfile(join(image_path1, f))]
image_files2 = [f for f in listdir(image_path2) if isfile(join(image_path2, f))]
image_files_test = [f for f in listdir(image_path_test) if isfile(join(image_path_test, f))]


In [6]:
len(scenes)

205

In [7]:
from meta_converter import MetaConverter
mc = MetaConverter("./")


In [8]:
def preprocess_image(): 


    # GENERIC VERSION OF GENERATING ORIGINAL DATA
    index2data={}
    object_dict={}
    index=0
    scene_folder = "simmc2_scene_jsons_dstc10_teststd"
    scenes = sorted(glob(str(scene_folder + "/*_scene.json")))
#     print(len(scenes))
    for scene in tqdm(scenes):
        if "wayfair" in scene:
            domain = "furniture"
        else:
            domain = "fashion"
        scene_data=load_json(scene)
        scene_objects = scene_data["scenes"][0]["objects"] #object lists

        scene_name = scene.replace("_scene.json","").split("/")[-1]
        scene_image = scene_name
    #     print(image_path1 + "/" + scene_image + ".png")
    #     print(scene_image)

        if scene_image[0]=="m":
            scene_image=scene_image[2:]
        if scene_image + ".png" in image_files_test:
            im = cv2.imread(image_path_test + "/" + scene_image + ".png")
#         if scene_image + ".png" in image_files1:
#             im = cv2.imread(image_path1 + "/" + scene_image + ".png")
#         elif scene_image + ".png" in image_files2:
#             im = cv2.imread(image_path2 + "/" + scene_image + ".png")
        else:
            print("there is no image available")
            continue

        # got image and scene data
        error_flag=False
        for obj in scene_objects:
#             print(obj)
            try:
                bbox = obj["bbox"]
                x=bbox[0]
                y=bbox[1]
                h=bbox[2]
                w=bbox[3]
                if h==0 or w==0:
                    raise Exception('bounding box size is zero!')
                # error if image has 
                crop_img = im[y:y+h, x:x+w]
                input_img = Image.fromarray(crop_img)
                meta_list = mc.convert_image(input_img,domain)

            except Exception as e:
                print(e)
                meta_list = []
                pass


            if scene_name not in object_dict:
                object_dict[scene_name]={}
            object_dict[scene_name][obj["index"]]=meta_list



    return object_dict

In [9]:
object_dict = preprocess_image()

 10%|▉         | 20/205 [00:08<00:57,  3.24it/s]

bounding box size is zero!


 20%|██        | 42/205 [00:17<01:07,  2.42it/s]

bounding box size is zero!


 31%|███       | 63/205 [00:26<01:05,  2.18it/s]

...

 77%|███████▋  | 157/205 [01:20<00:19,  2.45it/s]

bounding box size is zero!


 80%|███████▉  | 163/205 [01:23<00:17,  2.42it/s]

bounding box size is zero!


100%|██████████| 205/205 [01:36<00:00,  2.13it/s]


In [10]:
import gzip
import pickle

In [11]:
with gzip.open("object_100_threshold.pickle", 'w') as out:
  pickle.dump(object_dict,out)

In [12]:
with open("object_100_threshold.json", 'w') as out:
  json.dump(object_dict,out)

In [20]:
# len(image_dict)

NameError: name 'image_dict' is not defined

In [ ]:
# color_dict={}
# seen_colors={}
# for k,v in image_dict.items():
#     if len(v[1]["color"].split(","))==1:
#         color=v[1]["color"]
#         im=v[2]
#         if color not in seen_colors:
#             seen_colors[color]=0
#         seen_colors[color]+=1
#         color_dict[k]=[color,im]

In [ ]:
# len(color_dict)==32937

In [ ]:
seen_colors

In [13]:
# build directories for data
def build_color_dataset(image_dict):
    color_dict={}
    seen_colors={}
    for k,v in image_dict.items():
        if len(v[1]["color"].split(","))==1:
            color=v[1]["color"]
            im=v[2]
            if color not in seen_colors:
                seen_colors[color]=0
            seen_colors[color]+=1
            color_dict[k]=[color,im]   
    
    type_set=list(seen_colors.keys())
    
    DATA_DIR = Path(f'vision_data/color/color_data')
    ORG_DATA_DIR = Path(f'vision_data/color/original_color_data')
    DATASETS=["train","val"]
    for ds in DATASETS:
        for cls in type_set:
            (DATA_DIR / ds / cls).mkdir(parents=True, exist_ok=True)
    for cls in type_set:
        (ORG_DATA_DIR / cls).mkdir(parents=True, exist_ok=True)

        
    # save photo in the correct folder
    for key,value in color_dict.items():
        cv2.imwrite(str(ORG_DATA_DIR / value[0] / (key+".jpeg")), value[1])
            
    for i, ty in enumerate(type_set):
        image_paths = np.array(glob(f'{ORG_DATA_DIR}/{ty}/*.jpeg'))
        class_name = ty
#             print(f'{class_name}: {len(image_paths)}')
        np.random.shuffle(image_paths)

        ds_split = np.split(
        image_paths, 
        #     indices_or_sections=[int(.8*len(image_paths)), int(.9*len(image_paths))]
        indices_or_sections=[int(.9*len(image_paths))]

        )

        dataset_data = zip(DATASETS, ds_split)

        for ds, images in dataset_data:
            for img_path in images:
              shutil.copy(img_path, f'{DATA_DIR}/{ds}/{class_name}/')   

In [14]:
build_color_dataset(image_dict)

NameError: name 'image_dict' is not defined

In [ ]:
import json
filename="simmc2_dials_dstc10_train.json"
with open(filename,"r") as f:
    train_data = json.load(f)

In [ ]:
# 각 턴마다 최대 등장하는 오브젝트 개수.
numset=set()
for d in train_data["dialogue_data"]:
#     print(d.keys())
    for t in d["dialogue"]:
        numset.add(len(t["system_transcript_annotated"]["act_attributes"]["objects"]))
numset